In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import hashlib
import struct 
from collections import deque

In [2]:
import time

In [3]:
class JitterRNG:
    def __init__(self, pool_size=256):
        self.entropy_pool = deque(maxlen=pool_size)
        self.pool_size = pool_size
        self.last_time = None

    def _collect_timing_jitter(self, iterations=1000):
        jitter_data = []
        # TODO: First measurement to initialize
        self.last_time = time.perf_counter_ns()
        for _ in range(iterations):
                # TODO: Do some arbitrary computation to introduce variability
                dummy_var = 1
                for i in range(10): # A small number of iterations
                    dummy_var = (dummy_var * 1664525 + 1013904223) % (2**32) # Simple arithmetic

                # TODO: Measure the time again
                current_time = time.perf_counter_ns()

                # TODO: Calculate the jitter (timing difference)
                time_diff = current_time - self.last_time
                # TODO: Get the lowest 8 bits of time_diff
                lsb = time_diff & 0xFF 
                jitter_data.append(lsb)

                # TODO: update self.last_time for next iteration
                self.last_time = current_time
        return jitter_data

    def fill_entropy_pool(self):
        jitter_data = self._collect_timing_jitter(self.pool_size)
        for value in jitter_data:
            self.entropy_pool.append(value)
        return jitter_data

    def get_random_bytes(self, num_bytes=32):
        if len(self.entropy_pool) < self.pool_size:
            self.fill_entropy_pool()
        pool_bytes = bytes(self.entropy_pool)
        mixed_entropy = hashlib.sha256(pool_bytes).digest()
        result = bytearray()

        while len(result) < num_bytes:
            self.fill_entropy_pool()
            pool_bytes = bytes(self.entropy_pool)
            h = hashlib.sha256()
            h.update(mixed_entropy) 
            h.update(pool_bytes)   
            mixed_entropy = h.digest() 

            # Append the resulting hash bytes to our output buffer.
            result.extend(mixed_entropy)

        # Return only the requested number of bytes from the beginning of the result.
        return bytes(result[:num_bytes])
    
    def get_random_int(self, min_val=0, max_val=100):
        range_size = max_val - min_val + 1
        if range_size <= 0:
            raise ValueError("Invalid range: max_val must be greater than or equal to min_val")

        bits_needed = (range_size - 1).bit_length()
        if bits_needed == 0: 
             bits_needed = 1 

        bytes_needed = (bits_needed + 7) // 8

        random_bytes = self.get_random_bytes(bytes_needed)

        value = int.from_bytes(random_bytes, byteorder='big')
        return min_val + (value % range_size)
    
    def analyze_randomness(self, sample_size=1000):
        print(f"Generating {sample_size} samples for randomness analysis...")
        samples = []
        for _ in range(sample_size):
            samples.append(self.get_random_int(0, 255))

        print("Analysis complete. Plotting results.")

        plt.figure(figsize=(12, 10))

        plt.subplot(2, 2, 1)
        plt.hist(samples, bins=np.arange(0, 257, 8), color='blue', alpha=0.7, edgecolor='black')
        plt.title('Distribution of Random Values (0-255)')
        plt.xlabel('Value')
        plt.ylabel('Frequency')
        plt.grid(axis='y', alpha=0.5)

        plt.subplot(2, 2, 2)
        plt.plot(samples[:100], '.-', alpha=0.7)
        plt.title('First 100 Generated Values')
        plt.xlabel('Sample Index')
        plt.ylabel('Value (0-255)')
        plt.grid(True, alpha=0.5)

        plt.subplot(2, 2, 3)
        if len(samples) > 1:
            plt.scatter(samples[:-1], samples[1:], alpha=0.5, s=5)
            plt.title('Scatter Plot of Consecutive Values')
            plt.xlabel('Value n')
            plt.ylabel('Value n+1')
            plt.xlim(0, 255)
            plt.ylim(0, 255)
            plt.grid(True, alpha=0.5)
        else:
             plt.text(0.5, 0.5, 'Not enough samples for scatter plot', horizontalalignment='center', verticalalignment='center', transform=plt.gca().transAxes)

        plt.subplot(2, 2, 4)
        if len(samples) > 1:
            autocorr = np.correlate(samples, samples, mode='full')
            autocorr = autocorr[len(autocorr)//2:]
            autocorr = autocorr / autocorr[0]
            plt.plot(autocorr[:50])
            plt.title('Autocorrelation (First 50 Lags)')
            plt.xlabel('Lag')
            plt.ylabel('Correlation')
            plt.grid(True, alpha=0.5)
        else:
             plt.text(0.5, 0.5, 'Not enough samples for autocorrelation', horizontalalignment='center', verticalalignment='center', transform=plt.gca().transAxes)

        plt.tight_layout() 
        plt.show()

        return samples

In [4]:
# Example Usage:
# Create an instance of the JitterRNG
jitter_rng = JitterRNG(pool_size=256)

In [5]:
# Fill the entropy pool initially
print("Filling entropy pool...")
jitter_rng.fill_entropy_pool()
print(f"Entropy pool size: {len(jitter_rng.entropy_pool)}")

Filling entropy pool...
Entropy pool size: 256
